In [ ]:
# Cell 1: Setup and Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

# Set seeds for reproducibility
tf.random.set_seed(42)
print("TensorFlow version:", tf.__version__)

In [ ]:


# Cell 2: Configuration
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 10
MODEL_PATH = 'models/xray_cnn_model.h5'

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)
os.makedirs('sample_images', exist_ok=True)



In [ ]:
# Cell 3: Data Loading and Preprocessing
# Data paths (adjust based on your dataset structure)
train_dir = 'chest_xray/train'
val_dir = 'chest_xray/val'
test_dir = 'chest_xray/test'

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")



In [ ]:
# Cell 4: Model Architecture
def create_cnn_model():
    model = keras.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(2, 2),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        
        # Fourth Convolutional Block
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model

# Create and compile model
model = create_cnn_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Display model architecture
model.summary()




In [ ]:
# Cell 5: Training
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
    keras.callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# Save training history
with open('models/training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)



In [ ]:
# Cell 6: Evaluation and Visualization
# Evaluation
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"\n=== Test Results ===")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

# Save evaluation metrics
metrics = {
    'accuracy': float(test_accuracy),
    'precision': float(test_precision),
    'recall': float(test_recall),
    'f1_score': float(f1_score)
}

with open('models/evaluation_metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)



In [ ]:
# Cell 7: Training History Visualization
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    ax1.set_title('Model Accuracy Over Time')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    ax2.set_title('Model Loss Over Time')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('models/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history)



In [ ]:
# Cell 8: Sample Predictions
def save_sample_predictions(model, test_generator, num_samples=6):
    predictions = model.predict(test_generator)
    
    # Get sample images and their true labels
    test_generator.reset()
    sample_batch = next(test_generator)
    images, true_labels = sample_batch
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for i in range(min(num_samples, len(images))):
        axes[i].imshow(images[i])
        
        pred_prob = predictions[i][0]
        pred_class = "Pneumonia" if pred_prob > 0.5 else "Normal"
        true_class = "Pneumonia" if true_labels[i] == 1 else "Normal"
        
        axes[i].set_title(f"Prediction: {pred_class} ({pred_prob:.3f})\nActual: {true_class}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig('models/sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

save_sample_predictions(model, test_generator)



In [ ]:
# Cell 9: Model Architecture Diagram
keras.utils.plot_model(
    model, 
    to_file='models/model_architecture.png', 
    show_shapes=True, 
    show_layer_names=True,
    rankdir='TB',
    dpi=300
)

print("✅ Model training completed!")
print("✅ All artifacts saved in 'models/' directory")
print("✅ Ready for Streamlit deployment!")